<a href="https://colab.research.google.com/github/hemadamostafa/summoraize-with-bart/blob/main/Summarization_model_using_Bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.3 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.14.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.


In [2]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

In [3]:
# Loading tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
# Loading model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Please make sure the generation config includes `forced_bos_token_id=0`. 


Loading weights:   0%|          | 0/511 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Using device:", device)

Using device: cpu


In [6]:
import pandas as pd

In [11]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Set the path to your CSV file
file_path = '/content/drive/MyDrive/BBC_Data/bbc-text.csv'  # Adjust if your folder is somewhere else

# Step 3: Import pandas and read the CSV
import pandas as pd

df = pd.read_csv(file_path)

# Step 4: Check the first few rows
df.head()


Mounted at /content/drive


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [12]:
df.columns

Index(['category', 'text'], dtype='object')

In [13]:
articles = df["text"].tolist()

print("Number of articles:", len(articles))
print("\nSample article:\n")
print(articles[0][:500])

Number of articles: 2225

Sample article:

tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via h


In [14]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)   # normalize whitespace
    text = text.strip()
    return text

In [15]:
sample_text = clean_text(articles[0])

print(sample_text[:500])

tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home ne


In [16]:
text = clean_text(articles[0])

In [17]:
inputs = tokenizer(
    text,
    return_tensors="pt",
    max_length=1024,
    truncation=True
)

inputs = {key: value.to(device) for key, value in inputs.items()}

In [18]:
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=150,
    min_length=50,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

In [19]:
summary_text = tokenizer.decode(
    summary_ids[0],
    skip_special_tokens=True
)

In [20]:
print("ORIGINAL ARTICLE (first 500 chars):\n")
print(text[:500])

print("\n" + "="*80 + "\n")

print("GENERATED SUMMARY:\n")
print(summary_text)

ORIGINAL ARTICLE (first 500 chars):

tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home ne


GENERATED SUMMARY:

The way people watch tv will be radically different in five years time according to an expert panel which gathered at the annual consumer electronics show in las vegas. programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices.


In [21]:
def summarize_text(text, max_input_length=1024, max_summary_length=150):
    text = clean_text(text)

    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_input_length,
        truncation=True
    )

    inputs = {key: value.to(device) for key, value in inputs.items()}

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_summary_length,
        min_length=50,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True
    )

    return summary

In [22]:
sample_indices = [0, 10, 50]

for idx in sample_indices:
    original_text = articles[idx]
    summary = summarize_text(original_text)

    print(f"\nARTICLE {idx} (first 400 chars):\n")
    print(original_text[:400])

    print("\n" + "-"*80 + "\n")

    print("SUMMARY:\n")
    print(summary)

    print("\n" + "="*100 + "\n")


ARTICLE 0 (first 400 chars):

tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite past

--------------------------------------------------------------------------------

SUMMARY:

The way people watch tv will be radically different in five years time according to an expert panel which gathered at the annual consumer electronics show in las vegas. programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices.



ARTICLE 10 (first 400 chars):

berlin cheers for anti-nazi film a german movie about an anti-nazi resistance heroin

In [8]:
!pip install gradio

In [9]:
import gradio as gr

In [26]:
from transformers import pipeline
import gradio as gr

summarizer = pipeline(
    "text-generation",
    model="facebook/bart-large-cnn"
)

def gradio_summarize(text):
    if not text.strip():
        return "Please enter some text to summarize."

    output = summarizer(
        text,
        max_new_tokens=150,
        do_sample=False
    )

    return output[0]["generated_text"]


Loading weights:   0%|          | 0/316 [00:00<?, ?it/s]

BartForCausalLM LOAD REPORT from: facebook/bart-large-cnn
Key                                                       | Status     |  | 
----------------------------------------------------------+------------+--+-
model.encoder.layers.{0...11}.fc2.bias                    | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn.k_proj.weight     | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn.v_proj.weight     | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn.v_proj.bias       | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn.k_proj.bias       | UNEXPECTED |  | 
model.encoder.layers.{0...11}.fc1.weight                  | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn_layer_norm.bias   | UNEXPECTED |  | 
model.encoder.layers.{0...11}.fc2.weight                  | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn.out_proj.weight   | UNEXPECTED |  | 
model.encoder.layers.{0...11}.self_attn.out_proj.bias     | UNEXPECTED |  | 
model.encoder.laye

In [28]:
def gradio_summarize(input_text):
    if input_text.strip() == "":
        return "Please enter some text to summarize."
    return summarize_text(input_text)

In [29]:
demo = gr.Interface(
    fn=gradio_summarize,
    inputs=gr.Textbox(
        lines=15,
        placeholder="Paste a news article or long text here..."
    ),
    outputs=gr.Textbox(
        lines=6,
        label="Generated Summary"
    ),
    title="Abstractive Text Summarization with BART",
    description=(
        "This demo uses a pretrained BART encoder–decoder transformer "
        "to generate abstractive summaries from long-form text."
    )
)

In [30]:
demo.launch(
    share=True,
    inbrowser=True
)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe806dc982194bdc64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
